# Example how to use STAC, xarray and dask

This example shows how tu use [STAC](https://stacspec.org/en/about/) (Spatio-Temporal Asset Catalog), [xarray](https://docs.xarray.dev/en/stable/) and [Dask](https://www.dask.org/) for processing big raster datasets, also with good support for time series. The main idea is to first define the search and processing as process graph. The downloading and processing is done lazily at the end, so that only needed data (good enough cloud-free image, only needed bands and area) is downloaded. The libraries take care of data download, so you do not need to know about file paths. These tools work best when data is provided as [Cloud-optimized GeoTiffs](https://www.cogeo.org/) (COGs).

For trying out this example, it is recommended to start interactive [Jupyter session](https://docs.csc.fi/computing/webinterface/jupyter/) with [Puhti web interface](https://docs.csc.fi/computing/webinterface/), for example with 4 cores and 12 Gb memory.

Dask is used for parallization of computing, see [CSC Dask tutorial](https://docs.csc.fi/support/tutorials/dask-python/), inc how to use Dask with Jupyter in
Puhti web interface and how to create batch jobs with Dask.

We'll search for 12 months of Sentinel-2 data overlapping cetnral Helsinki. Then filter out cloudy scenes, based on their metadata, then create a median composite for each month.

The main steps:
* Start Dask cluster
* Query STAC catalogue to find Sentinel2 L2A images from area and time of interest and create first datacube.
* Removing images with too high cloud coverage.
* Selecting only required bands.
* Mosaic the images with median value, for each month.
* Select data only from exact area of interest.
* Finally, calculate the result.

In this example [Element84 STAC catalogue](https://www.element84.com/earth-search/) `sentinel-s2-l2a-cogs` collection on AWS is used, but there are several [other STAC catalogues available](https://stacspec.org/en/about/datasets/).

This example works with [geoconda module](https://docs.csc.fi/apps/geoconda/) in Puhti, the required libraries can be seen from imports.

The example is mostly based on [Stackstac documentation](https://stackstac.readthedocs.io/en/latest/basic.html)

In [ ]:
import stackstac
import pystac_client
import pyproj

Start Dask cluster. 

For following how Dask works open [Dask Dashboard or JupyterLab Dask Extension](https://docs.csc.fi/support/tutorials/dask-python/#dask-with-jupyter).

In [ ]:
from dask.distributed import Client

client = Client()
client

Define the center of area of interest, in this case Helsinki.

In [ ]:
lon, lat = 24.945, 60.173, 

Search from STAC API, using [pystac-client](https://pystac-client.readthedocs.io/). If using some other STAC catalogue, change the URL. 

In [ ]:
# Define STAC API URL and create
URL = "https://earth-search.aws.element84.com/v0"
catalog = pystac_client.Client.open(URL)

Find out which collections are available.

In [ ]:
for collection in catalog.get_collections():
    print(collection.id)

Define search critera, here location, collection (`sentinel-s2-l2a-cogs`) and time period. The results provide metadata about the relevant scenes, and links to their data.

In [ ]:
%%time
items = catalog.search(
    intersects=dict(type="Point", coordinates=[lon, lat]),
    collections=["sentinel-s2-l2a-cogs"],
    datetime="2020-01-01/2020-03-01"
).get_all_items()
len(items)

Create `xarray` datacube from the items. Using all the defaults, our data will be in its native coordinate reference system, at the finest resolution of all the assets. This will be fast, because the actual data is not fetched yet. 

In [ ]:
%time stack = stackstac.stack(items)

How does the datacube look like?

In [ ]:
stack

Filter out scenes with >20% cloud coverage (according to the `eo:cloud_cover` field set by the data provider).
Then, pick the bands corresponding to red, green, and blue, and use xarray's `resample` to create 1-month median composites.

In [ ]:
lowcloud = stack[stack["eo:cloud_cover"] < 20]
rgb = lowcloud.sel(band=["B04", "B03", "B02"])
monthly = rgb.resample(time="MS").median("time", keep_attrs=True)

With these limitation the amount of data has decreased from 2 TB to ~30 Gb.

In [ ]:
monthly

Convert lat-lon point to the data's UTM coordinate reference system, then use that to slice the `x` and `y` dimensions, which are indexed by their UTM coordinates.

In [ ]:
x_utm, y_utm = pyproj.Proj(monthly.crs)(lon, lat)
buffer = 2000  # meters

aoi = monthly.loc[..., y_utm+buffer:y_utm-buffer, x_utm-buffer:x_utm+buffer]
aoi

So far no data has been downloaded, nor anything computed with actual data. Data size has become 40 Mb, which will actually be downloaded. In this example the final data size is very small, but Dask is good also in handling much bigger amounts of data, also bigger than fits to memory.

To start the process use `compute()`. The process can be followed from Dask Dashboard or Dask Lab Extension.

In [ ]:
%%time
data = aoi.compute()

Show the resulting images.

In [ ]:
data.plot.imshow(row="time", rgb="band", robust=True, size=10);